In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Create an app

*Apps let you take the results inside your analysis (such as plots and datasets), and turn them into an interactive web applications which you can share with anyone.*

## Introduction

You have analyzed data and uncovered some insights in Jupyter, and you now need to share this with a team-member or stakeholder.

Datapane makes this simple by allowing you to assemble the plots, data, and code in your notebook into an **app**, which you can share with end-users. Your users can access and use your app using their browser, without understanding or installing Python and Jupyter.

Datapane allows you to create apps in a few seconds using the objects you already have in your Python analyses, such as pandas DataFrames, plots from visualization libraries, and Markdown text. Additionally, it provides a set of interactive app components, such as dropdowns, tabs, and pages which allow you to create rich, interactive experiences for your end-users.

## A simple app

Apps are comprised of a collection of [data-centric blocks](/tutorials/blocks). For instance, Datapane provides a `DataTable` block that takes a pandas `DataFrame`, and displays it as an interactive table in your app. 

The first step to building and sharing an app is importing datapane's Python library in your Jupyter Notebook or script.

Next, we can create a `DataTable` block by passing a `DataFrame` into it, and create a `View` with that single block in it.

In [ ]:
import pandas as pd
import datapane as dp

df = pd.read_csv(
    "https://covid.ourworldindata.org/data/vaccinations/vaccinations-by-manufacturer.csv",
    parse_dates=["date"],
)
df = (
    df.groupby(["vaccine", "date"])["total_vaccinations"].sum().tail(1000).reset_index()
)

v = dp.View(dp.DataTable(df))

We can now render this app within a Jupyter notebook.

In [ ]:
v

Or save it to a local `HTML` file for testing.

In [ ]:
dp.save_report(v, path="simple-app.html")

Copying this code into a notebook or new script and running it will generate the app.

!!! Sharing

    Local HTML files are good for testing, but the best way to share an app is to host it on the web. If you upload your app to [_Datapane Cloud_](/tutorials/datapane-cloud/upload-an-app-to-datapane-cloud/), you will have a URL which you can share with your audience, who will be able to explore and download your dataset.

## A richer app

That app was pretty basic, but we can jazz it up by adding some plots and Markdown text. Unlike a BI tool, Datapane does not rely on a proprietary visualization engine; instead, it natively supports Python visualization libraries such as [Altair](https://altair-viz.github.io/), [Plotly](https://plotly.com/python/), [Bokeh](https://bokeh.org/), and [Folium](https://python-visualization.github.io/folium/).

Let's take the example above, and plot some data using the Python library Altair and add some text.

!!! info

    We'll use the below sample code snippet for the rest of the tutorials, so feel free to copy and paste it into a new Python script or Jupyter notebook and follow along.

In [ ]:
import pandas as pd
import altair as alt
import datapane as dp

# download data & group by manufacturer
df = pd.read_csv(
    "https://covid.ourworldindata.org/data/vaccinations/vaccinations-by-manufacturer.csv",
    parse_dates=["date"],
)
df = (
    df.groupby(["vaccine", "date"])["total_vaccinations"].sum().tail(1000).reset_index()
)

# plot vaccinations over time using Altair
plot = (
    alt.Chart(df)
    .mark_area(opacity=0.4, stroke="black")
    .encode(
        x="date:T",
        y=alt.Y("total_vaccinations:Q"),
        color=alt.Color("vaccine:N", scale=alt.Scale(scheme="set1")),
        tooltip="vaccine:N",
    )
    .interactive()
    .properties(width="container")
)

# tablulate total vaccinations by manufacturer
total_df = (
    df[df["date"] == df["date"].max()]
    .sort_values("total_vaccinations", ascending=False)
    .reset_index(drop=True)
)
total_styled = total_df.style.bar(
    subset=["total_vaccinations"],
    color="#5fba7d",
    vmax=total_df["total_vaccinations"].sum(),
)

# embed into a Datapane app
v = dp.View(
    "## Vaccination App",
    dp.Plot(plot, caption="Vaccinations by manufacturer over time"),
    dp.Table(total_styled, caption="Current vaccination totals by manufacturer"),
    dp.DataTable(df, caption="Initial Dataset"),
)

Like before, can render this app within a Jupyter notebook.

In [ ]:
v

Or save it to a local `HTML` file for testing.

In [ ]:
dp.save_report(v, path="richer-app.html")

Next, we will explore the blocks that make up an app, followed by how to upload and share apps online.